In [4]:
import os, sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: e:\antibody-seq-developability\antibody-seq-developability


In [5]:
from src.data_oas import load_oas_human_paired

df = load_oas_human_paired()
df.head()
df.shape


Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

(1692921, 9)

In [ ]:
from src.data_oas import load_oas_human_paired
from src.features_cdr import compute_cdr3_features

df = load_oas_human_paired()
df_feat = compute_cdr3_features(df)
df_feat.head()
